In [1]:
import pandas as pd
import numpy as np

import boto3
import io

import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


# Set the maximum number of rows and Columns to 200
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)

In [2]:
# Create function to load s3 files from AWS
def load_s3_file(file_name, s3, bucket_name, df_name):
    # Downloading the file from S3 and parsing it into a dataframe
    s3.download_file(bucket_name, file_name, file_name)
    
    # Low memory is set to False because columns have mixed data types
    df_name = pd.read_csv(file_name, index_col=False, low_memory=False)
    return df_name 

In [3]:
# Load files from AWS
s3R = boto3.resource("s3")
s3 = boto3.client("s3")
bucket_name = "capstonehaystacks"

# Listing the contents of the S3 bucket
for obj in s3R.Bucket(bucket_name).objects.all():
    print(obj.key)

ACSDP5Y2021.DP04-Data.csv
ACSST5Y2021.S0101-Data.csv
ACSST5Y2021.S0801-Data.csv
ACSST5Y2021.S1901-Data.csv
GA_LISTINGS_SALES.csv
GA_LISTINGS_SALES_V2.csv
GA_listing.csv
POI_second_tab.csv
all_zips_grocery_store.json
all_zips_restaurant.json
atlanta-geo.csv
atlanta_cbsa_zip.csv
atlanta_hdma_2021.csv
atlanta_hdma_census_2021.csv
atlanta_hdma_tract_2021.csv
atlanta_listings.csv
census_all.csv
census_all_perCapita.csv
census_data
core_geo_dataset.csv
crime.csv
crime_rating_zipcode.csv
elementary_schools.csv
ga_georgia_zip_codes_geo.min.json
high_schools.csv
hmda_2017_ga_all-records_labels.csv
hmda_2021_ga_all-records_labels.csv
jason_listing.csv
listing-with-poi-distances.csv
listings_with_tract.csv
middle_schools.csv
poi-google-exploded.csv
poi_combined_haystack.csv
poi_combined_haystack_ALL.csv
poi_combined_haystack_ALL_CLEANED.csv
poi_expanded.csv
points-of-interest-google.csv
points-of-interest-google2.csv
points-of-interest-google3.csv
points-of-interest-haystacks.csv
schools.csv
zip_

In [33]:
# Load S3 files to variables
census = load_s3_file("census_all.csv", s3, bucket_name, df_name="census")
census_capita = load_s3_file("census_all_perCapita.csv", s3, bucket_name, df_name="census_capita")
hdma = load_s3_file("atlanta_hdma_2021.csv", s3, bucket_name, df_name="hdma")
POI = load_s3_file("POI_second_tab.csv", s3, bucket_name, df_name="POI")

In [ ]:
hdma.columns

In [ ]:
census_capita.columns

### DROP some Census Columns

In [5]:
census_capita.columns[:50]

Index(['zipcode', 'restaurant_above_4.7_percentage', 'total_workers',
       'car_commute', 'travel_less_10', 'travel_10_14', 'travel_15_19',
       'travel_20_24', 'travel_25_29', 'travel_30_34', 'travel_35_44',
       'travel_45_59', 'travel_more_60', 'travel_mean', 'population',
       'percent_male', 'percent_under_15', 'percent_teen_15_19',
       'percent_college_20_24', 'percent_25_39', 'percent_40-59',
       'percent_over_60', 'total_units', 'rental_vacancy_rate',
       'median_homeowner_value', 'median_rental_value',
       'percent_owner_occupied', 'percent_after_2019',
       'rent_less_15_percent_income', 'rent_over_30_percent',
       'rent_15_30_percent', 'rent_less_999', 'rent_1000_2500',
       'rent_over_2500', 'gross_rental_yield', 'total_households',
       'percent_less_10k', 'percent_10k_15k', 'percent_15k_25k',
       'percent_25k_35k', 'percent_35k_50k', 'percent_50k_75k',
       'percent_75k_100k', 'percent_100k_150k', 'percent_150k_200k',
       'percent_more

In [6]:
drop_cols = ['total_workers', 'total_households', 'household_median_income',
             'restaurant_above_4.7_percentage', 'restaurant_above_4.7_per_Capita', 'restaurant_below_4.7_per_Capita']
census_capita = census_capita.drop(drop_cols, axis=1)

### Standardizing Values in Census Data/POI

In [ ]:
# Select the column to exclude
column_to_exclude = 'zipcode'

df_to_standardize = census_capita.drop(column_to_exclude, axis=1)
scaler = StandardScaler()
standardized_values = scaler.fit_transform(df_to_standardize)
census_capita_standardized = pd.DataFrame(standardized_values, columns=df_to_standardize.columns)

# Add the excluded column back to the standardized DataFrame
census_capita_standardized[column_to_exclude] = census_capita[column_to_exclude]

# Pop the excluded column and insert it at the first position
zipcode_column = census_capita_standardized.pop(column_to_exclude)
census_capita_standardized.insert(0, column_to_exclude, zipcode_column)

census_capita_standardized.head()

#### Reverse Standardization (not neccesary)

In [ ]:
reversed_values = scaler.inverse_transform(standardized_values)

# Create a new DataFrame with reversed values
census_capita_reversed = pd.DataFrame(reversed_values, columns=df_to_standardize.columns)

# Add the excluded column back to the reversed DataFrame
census_capita_reversed[column_to_exclude] = census_capita[column_to_exclude]

# Pop the excluded column and insert it at the first position
zipcode_column = census_capita_reversed.pop(column_to_exclude)
census_capita_reversed.insert(0, column_to_exclude, zipcode_column)

census_capita_reversed.head()

### KNN to find Similar Profiled Zipcodes

In [ ]:
# Feature Selection
selected_cols = census_capita_standardized.drop('zipcode', axis=1).columns ## Investor Profile Builder
X = census_capita_standardized.drop('zipcode', axis=1).values 

k=6 # No. of zipcodes to consider
model = NearestNeighbors(n_neighbors=k)
model.fit(X)

In [ ]:
# Define my Zip Code vals
my_zip = 30002
my_zip_vals = census_capita_standardized[census_capita_standardized['zipcode'] == my_zip].iloc[:,1:]
my_zip_vals

In [ ]:
# Find Similar Zip Codes
new_X = my_zip_vals.values

distances, indices = model.kneighbors(new_X)
similar_zip_codes = census_capita_standardized.iloc[indices[0]]['zipcode']  

print("Similar Zip Codes:")
similar_zip_codes

In [ ]:
distances

In [ ]:
my_sim_zips = census_capita[census_capita['zipcode'].isin(similar_zip_codes)]
my_sim_zips

### Cosine Similarity for Census, HDMA, and POI data on selected ZIPCODE

In [ ]:
# Define Column subsets
census_col_names = my_sim_zips.iloc[:,1:43].columns.tolist()
poi_col_names = my_sim_zips.iloc[:,44:].columns.tolist()

In [ ]:
census_sim = ['zipcode']
poi_sim = ['zipcode']

# Loop through each column name subset
for col_names, sim_list in zip([census_col_names, poi_col_names], [census_sim, poi_sim]):
    # Select the subset of columns
    selected_cols = my_sim_zips[col_names]
    similar_rows = selected_cols.iloc[1:]

    # Calculate pairwise similarity matrix for the selected rows using cosine similarity
    similarity_matrix = cosine_similarity(similar_rows.T)
    similarity_df = pd.DataFrame(similarity_matrix, index=selected_cols.columns, columns=similar_rows.columns)

    # Sort the columns based on similarity scores
    sorted_columns = similarity_df.mean(axis=1).sort_values(ascending=False)
    top_similar_cols = sorted_columns[:5].index.tolist()

    # Assign the top similar columns to the respective list
    sim_list.extend(top_similar_cols)


print("Top similar columns for the Census subset:")
print(census_sim)
print("\n")

print("Top similar columns for the POI subset:")
print(poi_sim)
print("\n")

In [ ]:
my_sim_zips[census_sim]

In [ ]:
my_sim_zips[poi_sim]

In [ ]:
## Maybe for POI, it makes the most sense to look into what is the MOST poi's are in that zipcode 
## and see which of those similar zipcodes has a similar makeup

#### Combine into a dataframe

In [ ]:
sim_zip_table = pd.merge(left= my_sim_zips[census_sim], right=my_sim_zips[poi_sim], how='left', on='zipcode' ) 
sim_zip_table['zip_cat'] = np.where(sim_zip_table['zipcode'] == my_zip, 'My_Zip', 'Similar_Zip')
# Pop the excluded column and insert it at the first position
zipcode_column = sim_zip_table.pop('zip_cat')
sim_zip_table.insert(0, 'zip_cat', zipcode_column)
sim_zip_table

----------------

### Create Zipcode Function 

In [72]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

def sim_zip(zipcode, df, columns, POI_df, k=6, mode=1):
    '''
    Finds similar profiled zip codes to the user-entered zipcode and performs different operations based on the mode.

    Parameters:
        zipcode (int): User-entered zipcode.
        df (DataFrame): DataFrame with the zipcode column named 'zipcode' in the first index and various features.
        columns (list): List of column names to select.
        POI_df (DataFrame): DataFrame with point of interest data.
        k (int): Number of zipcodes to consider. Default is 6.
        mode (int): Mode of operation. Valid values are 1, 2, 3, or 4.

    Returns:
        - If mode is 1: Series with k additional similar profiled zipcodes based on KNN.
        - If mode is 2: DataFrame with with all df columns retained for reference.
        - If mode is 3: Series with top 5 point of interest names in similar profiled zipcodes.
        - If mode is 4: Series with top 5 primary categories of point of interests in similar profiled zipcodes.

    Raises:
        ValueError: If the zipcode value is not an integer, df doesn't have 'zipcode' column as the first column,
                    or if the mode is not 1, 2, 3, or 4.
    '''
    # Check if the zipcode is a number
    if not isinstance(zipcode, int):
        raise ValueError('The zipcode value must be an integer.')

    # Check if 'zipcode' is the first column in df
    if df.columns[0] != 'zipcode':
        raise ValueError("The 'zipcode' column must be the first column in the dataframe.")

    # Check if mode is valid
    valid_modes = [1, 2, 3, 4]
    if mode not in valid_modes:
        raise ValueError(f"Invalid mode value. Valid modes are: {valid_modes}")

    # Standardize Values
    column_to_exclude = 'zipcode'  # I don't want to standardize zipcode
    df_to_standardize = df[columns]
    scaler = StandardScaler()
    standardized_values = scaler.fit_transform(df_to_standardize)
    census_capita_standardized = pd.DataFrame(standardized_values, columns=df_to_standardize.columns)

    # Add Zipcode column back
    census_capita_standardized[column_to_exclude] = df[column_to_exclude]

    # Pop Zipcode to the first position
    zipcode_column = census_capita_standardized.pop(column_to_exclude)
    census_capita_standardized.insert(0, column_to_exclude, zipcode_column)

    # KNN
    X = census_capita_standardized.drop('zipcode', axis=1).values
    model = NearestNeighbors(n_neighbors=k)
    model.fit(X)

    # Filter df to my Zip Code
    my_zip_vals = census_capita_standardized[census_capita_standardized['zipcode'] == zipcode].iloc[:, 1:]

    # Find Similar Zip Codes
    new_X = my_zip_vals.values
    distances, indices = model.kneighbors(new_X)
    similar_zip_codes = census_capita_standardized.iloc[indices[0]]['zipcode']

    # Filter df by sim zips
    my_sim_zips = df[df['zipcode'].isin(similar_zip_codes)]

    if mode == 1:
        return my_sim_zips
    elif mode == 2:
        return my_sim_zips
    elif mode == 3:
        return POI_df[POI_df['zipcode'].isin(similar_zip_codes)].groupby(by='name').size().sort_values(ascending=False)[:5]  
    elif mode == 4:
        return POI_df[POI_df['zipcode'].isin(similar_zip_codes)].groupby(by='primary_category').size().sort_values(ascending=False)[:5]
    
    

In [71]:
zipcode= 30002
df=census_capita
columns=['travel_less_10', 'percent_25_39', 'household_mean_income', 
           'percent_100k_150k', 'rental_vacancy_rate']

sim_zip(30002, census_capita,columns, POI_df=POI, k=10, mode =1)

0      30002
38     30067
174    30360
172    30350
167    30341
45     30080
44     30079
46     30082
26     30044
69     30126
Name: zipcode, dtype: int64

In [73]:
sim_zip(30002, census_capita,columns, POI_df=POI, k=10, mode =2)

zipcode  restaurant_above_4.7_percentage  total_workers  car_commute  \
0      30002                        11.111111           3221         77.1   
26     30044                         9.523810          47288         88.0   
38     30067                         7.692308          24362         76.6   
44     30079                              NaN           1868         71.7   
45     30080                         6.024096          31269         77.7   
46     30082                        11.111111          14582         75.5   
69     30126                        12.121212          21822         82.1   
167    30341                         9.523810          17861         74.6   
172    30350                        12.500000          22896         70.5   
174    30360                        13.333333           8065         79.1   

     travel_less_10  travel_10_14  travel_15_19  travel_20_24  travel_25_29  \
0               5.1           9.6          14.1          15.7           3.6   
26              4.3          10.2          15.5          13.8           5.0   
38              6.6           9.9          14.2          14.4           6.3   
44              5.7          18.0           3.4          14.9           5.8   
45              6.6          12.4          12.6          14.4           9.6   
46              4.1           7.1          15.3          11.9           6.8   
69              3.1           6.6          12.0          15.4           6.5   
167             4.7           7.3          23.3          20.7           8.0   
172             5.1          11.0          18.0          17.6           9.3   
174             6.1          10.6          14.9          14.6           8.3   

     travel_30_34  travel_35_44  travel_45_59  travel_more_60  travel_mean  \
0            24.0          10.2           8.8             8.9         29.7   
26           13.7           9.2          12.8            15.4         32.4   
38           25.7           7.5           8.6             6.9         27.8   
44           21.3           2.7           9.9            18.4         30.5   
45           18.9           9.2           9.9             6.4         26.8   
46           24.3           8.2          13.3             9.0         31.0   
69           19.7          12.4          10.9            13.4         32.5   
167          15.0           8.6           9.0             3.5         25.5   
172          17.8           5.1           8.3             7.7         28.8   
174          16.3          11.8          10.2             7.2         27.9   

     population  percent_male  percent_under_15  percent_teen_15_19  \
0          6025          49.0              19.4                 1.6   
26        99613          50.0              24.7                 7.6   
38        45584          51.0              19.2                 5.7   
44         4042          48.0              30.0                 6.4   
45        53270          48.0              18.6                 3.6   
46        26161          51.0              19.3                 4.3   
69        43134          48.0              21.0                 6.1   
167       33131          50.0              20.5                 6.8   
172       37635          49.0              14.3                 5.4   
174       15866          49.0              24.0                 6.4   

     percent_college_20_24  percent_25_39  percent_40-59  percent_over_60  \
0                      2.1           25.4           27.6             24.1   
26                     6.6           22.1           25.6             13.5   
38                     7.8           29.6           21.9             15.8   
44                     5.1           22.5           21.7             14.2   
45                     5.3           30.7           27.4             14.3   
46                     4.6           26.3           28.1             17.5   
69                     4.4           24.2           27.4             16.9   
167                    6.7    

In [74]:
sim_zip(30002, census_capita,columns, POI_df=POI, k=10, mode =3)

name
Shell                23
Subway               23
Metro by T-Mobile    18
Waffle House         16
McDonald's           15
dtype: int64

In [75]:
sim_zip(30002, census_capita,columns, POI_df=POI, k=10, mode =4)

primary_category
Auto repair shop     109
Apartment complex    103
Gas station          103
Restaurant            95
Hair salon            87
dtype: int64

----

In [ ]:
# def sim_zip_OLD(zipcode, df, k=6, col_break=43):
#     '''
#     Finds similar profiled zip codes to the user-entered zipcode.

#     Parameters:
#         zipcode (int): User-entered zipcode.
#         df (DataFrame): DataFrame with the zipcode column named 'zipcode' in the first index and various features.
#         k (int): Number of zipcodes to consider. Default is 6.
#         col_break (int): Index position to break the columns into subsets. Default is 43.

#     Returns:
#         DataFrame: DataFrame with similar profiled zipcodes and selected columns.
    
#     Raises:
#         ValueError: If the zipcode value is not a number.

#     '''
#     # Check if the zipcode is a number
#     if not isinstance(zipcode, int):
#         raise ValueError('The zipcode value must be an integer.')
        
#     # StandardScalar
#     column_to_exclude = 'zipcode'
#     df_to_standardize = df.drop(column_to_exclude, axis=1)
#     scaler = StandardScaler()
#     standardized_values = scaler.fit_transform(df_to_standardize)
#     census_capita_standardized = pd.DataFrame(standardized_values, columns=df_to_standardize.columns)
#     ## Add the excluded column back to the standardized DataFrame
#     census_capita_standardized[column_to_exclude] = census_capita[column_to_exclude]
#     ## Pop the excluded column and insert it at the first position
#     zipcode_column = census_capita_standardized.pop(column_to_exclude)
#     census_capita_standardized.insert(0, column_to_exclude, zipcode_column)

#     # KNN
#     selected_cols = census_capita_standardized.drop('zipcode', axis=1).columns
#     X = census_capita_standardized.drop('zipcode', axis=1).values 
#     ## k= No. of zipcodes to consider
#     model = NearestNeighbors(n_neighbors=k)
#     model.fit(X)

#     # filter df to my Zip Code
#     my_zip_vals = census_capita_standardized[census_capita_standardized['zipcode'] == zipcode].iloc[:,1:]

#     # Find Similar Zip Codes
#     new_X = my_zip_vals.values
#     distances, indices = model.kneighbors(new_X)
#     similar_zip_codes = census_capita_standardized.iloc[indices[0]]['zipcode']  

#     # Filter df by sim zips
#     my_sim_zips = df[df['zipcode'].isin(similar_zip_codes)]

#     # Define Column subsets
#     census_col_names = my_sim_zips.iloc[:, 1:colbreak].columns.tolist()
#     poi_col_names = my_sim_zips.iloc[:, colbreak+1:].columns.tolist()

#     # Define Empty Lists to populate
#     census_sim = ['zipcode']
#     poi_sim = ['zipcode']

#     # Loop through each column name subset
#     for col_names, sim_list in zip([census_col_names, poi_col_names], [census_sim, poi_sim]):
#         # Select the subset of columns
#         selected_cols = my_sim_zips[col_names]
#         similar_rows = selected_cols.iloc[1:]

#         # Calculate pairwise similarity matrix for the selected rows using cosine similarity
#         similarity_matrix = cosine_similarity(similar_rows.T)
#         similarity_df = pd.DataFrame(similarity_matrix, index=selected_cols.columns, columns=similar_rows.columns)

#         # Sort the columns based on similarity scores
#         sorted_columns = similarity_df.mean(axis=1).sort_values(ascending=False)
#         top_similar_cols = sorted_columns[:5].index.tolist()

#         # Assign the top similar columns to the respective list
#         sim_list.extend(top_similar_cols)
       
#     # Create Combined Table 
#     sim_zip_table = pd.merge(left= my_sim_zips[census_sim], right=my_sim_zips[poi_sim], how='left', on='zipcode' ) 
#     # New col to seperate my zip from similar zips
#     sim_zip_table['zip_cat'] = np.where(sim_zip_table['zipcode'] == zipcode, 'My_Zip', 'Similar_Zip')
#     # Pop the excluded column and insert it at the first position
#     zipcode_column = sim_zip_table.pop('zip_cat')
#     sim_zip_table.insert(0, 'zip_cat', zipcode_column)
#     return sim_zip_table
